# Setu p

In [1]:
import os
import sys
import numpy as np
import pandas as pd

np.random.seed(40)  # Set the seed for reproducibility

# Check if the original directory is already saved in a variable
if 'original_directory' not in globals():
    # Save the original working directory the first time
    original_directory = os.getcwd()

# Change back to the original directory whenever the cell is executed
os.chdir(original_directory)

# Go to mother directory
os.chdir("../")

# Verify the current working directory
print("Working directory set to:", os.getcwd())

sys.path.append(os.path.abspath(os.getcwd()))

Working directory set to: c:\Users\Biebert\OneDrive - Universität St.Gallen\Dokumente\OneDrive Dokumente\02_Bildung\01_BVWL Bachelor\01_Kurse\07_Thesis\Code\Portfolio_Optimization_DDPG


# Loading and combining

## Portfolio

In [42]:
reward_function = 'dsr'
# 9904_wealth_over_time_20241029_232625_diff_sharpe_reward

# 9904_actions_taken_20241029_232625_diff_sharpe_reward
# 9102_actions_taken_20241101_210054_diff_sharpe_reward

# 9904_episode_scores_20241029_232625_diff_sharpe_reward
# 9102_episode_scores_20241101_210054_diff_sharpe_reward

# 9904_rewards_received_20241029_232625_diff_sharpe_reward
# 9102_rewards_received_20241101_210054_diff_sharpe_reward

# 9904_states_observed_20241029_232625_diff_sharpe_reward
# 9102_states_observed_20241101_210054_diff_sharpe_reward

# 9904_wealth_over_time_20241029_232625_diff_sharpe_reward
# 9102_wealth_over_time_20241101_210054_diff_sharpe_reward


first_df = pd.read_csv(f'data_save/9904_wealth_over_time_20241029_232625_diff_sharpe_reward.csv')
second_df = pd.read_csv(f'data_save/9102_wealth_over_time_20241101_210054_diff_sharpe_reward.csv')

In [43]:
max_episode_df1 = first_df['Episode'].max()
second_df_changed = second_df.copy()
second_df_changed['Episode'] += max_episode_df1 + 1

final_df = pd.concat([first_df, second_df_changed], ignore_index=True)


In [44]:
print(final_df.shape)

print(final_df.shape[0]/750)

final_df

(6001500, 4)
8002.0


,Episode,TimeStep,Simulation,Wealth
0,0,0,16,1006.852172
1,0,1,16,1019.873519
2,0,2,16,1027.369734
3,0,3,16,1020.719864
4,0,4,16,1029.807389
...,...,...,...,...
6001495,8001,745,329,1442.118369
6001496,8001,746,329,1446.958891
6001497,8001,747,329,1450.911904
6001498,8001,748,329,1456.081671


In [45]:
# save the data
# portfolio_value_action, portfolio_value_score, portfolio_value_rewards, portfolio_value_states, portfolio_value_wealth
final_df.to_csv('00_final_data/dsr_wealth.csv', index=False)

## DSR

In [1]:
# Garch
import os
import sys
import numpy as np
import pandas as pd

np.random.seed(40)  # Set the seed for reproducibility

# Check if the original directory is already saved in a variable
if 'original_directory' not in globals():
    # Save the original working directory the first time
    original_directory = os.getcwd()

# Change back to the original directory whenever the cell is executed
os.chdir(original_directory)

# Go to mother directory
os.chdir("../")

# Verify the current working directory
print("Working directory set to:", os.getcwd())

sys.path.append(os.path.abspath(os.getcwd()))


Working directory set to: c:\Users\Biebert\OneDrive - Universität St.Gallen\Dokumente\OneDrive Dokumente\02_Bildung\01_BVWL Bachelor\01_Kurse\07_Thesis\Code\Portfolio_Optimization_DDPG


In [66]:
from data_scripts import generate_data as gd
from data_scripts import plotting as pl
import yfinance as yf
from arch import arch_model

np.random.seed(40) 

tickers = ['MSFT', 'TGT', 'QCOM', 'MU', 'CAT']
num_simulation_episodes = 500
num_simulation_days = 750


data = yf.download('MSFT', start="2000-01-01", end="2020-01-01")['Adj Close']
log_returns = np.log(data / data.shift(1)).dropna()
garch = arch_model(log_returns, p=1, q=1, dist='normal')
res = garch.fit(disp="off")
simulation = garch.simulate(res.params, 750)


log_returns_2 = np.log(data / data.shift(1)).dropna()*100
garch_2 = arch_model(log_returns_2, p=1, q=1, dist='normal')
res_2 = garch_2.fit(disp="off")
simulation_2 = garch_2.simulate(res_2.params, 750)

simulated_returns = np.exp(simulation) - 1
simulated_prices = 100 * (1 + simulated_returns).cumprod()

simulation_2 = simulation_2 * 0.01
simulated_returns_2 = np.exp(simulation_2) - 1
simulated_prices_2 = 100 * (1 + simulated_returns_2).cumprod()


# Combine in a df
final_df = pd.DataFrame()
final_df['Not Scaled'] = simulated_prices['data']
final_df['Scaled'] = simulated_prices_2['data']
final_df


[*********************100%%**********************]  1 of 1 completed
c:\Users\Biebert\miniconda3\envs\dsf\lib\site-packages\arch\univariate\base.py:311: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003602. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.




,Not Scaled,Scaled
0,101.849624,98.230812
1,105.345098,98.820624
2,103.501132,97.928631
3,103.747101,97.810558
4,103.360893,97.700017
...,...,...
745,244.143014,56.544688
746,243.634267,56.504035
747,238.166306,56.306178
748,236.368232,58.510077


In [77]:
# load data
MSFT = pd.read_csv('data/sim_test_MSFT.csv')
CAT = pd.read_csv('data/sim_test_CAT.csv')
MU = pd.read_csv('data/sim_test_MU.csv')
QCOM = pd.read_csv('data/sim_test_QCOM.csv')
TGT = pd.read_csv('data/sim_test_TGT.csv')

data = yf.download('MSFT', start="2000-01-01", end="2020-01-01")['Adj Close']
data

[*********************100%%**********************]  1 of 1 completed


Date
2000-01-03     35.935879
2000-01-04     34.721947
2000-01-05     35.088032
2000-01-06     33.912666
2000-01-07     34.355843
                 ...    
2019-12-24    150.833008
2019-12-26    152.069321
2019-12-27    152.347275
2019-12-30    151.034256
2019-12-31    151.139679
Name: Adj Close, Length: 5031, dtype: float64

In [86]:
tickers = ['MSFT', 'TGT', 'QCOM', 'MU', 'CAT']

for ticker in tickers:
    data = yf.download(ticker, start="2000-01-01", end="2020-01-01")['Adj Close']
    First = data[0]
    Last = data[-1]
    num_years = len(data) / 252
    real_return = ((Last / First) ** (1 / num_years)) - 1
    print('The real return for', ticker, 'is:', real_return)

    # Calculate cumulative returns from price columns
    returns_simulation = []
    sim_data = pd.read_csv(f'data/sim_test_{ticker}.csv')
    for col in sim_data.columns[1:]:
        sim_years = len(sim_data[col]) / 252
        First = sim_data[col].iloc[0]
        Last = sim_data[col].iloc[-1]
        cumulative_return = (Last / First) - 1
        annualized_sim_return = (1 + cumulative_return) ** (1 / sim_years) - 1
        returns_simulation.append(annualized_sim_return)
    print('The sim return for', ticker, 'is:', sum(returns_simulation) / len(returns_simulation))




[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


The real return for MSFT is: 0.07460365123878154
The sim return for MSFT is: 0.20154444197130814
The real return for TGT is: 0.0855483643866124
The sim return for TGT is: 0.12914334739239866


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


The real return for QCOM is: 0.016661776677028728
The sim return for QCOM is: 0.2050457644420627
The real return for MU is: 0.017590978688470038
The sim return for MU is: 0.20699673085218404


[*********************100%%**********************]  1 of 1 completed


The real return for CAT is: 0.12427920549091054
The sim return for CAT is: 0.2815282465352191


In [82]:
data = yf.download(ticker, start="2000-01-01", end="2020-01-01")['Adj Close']
data[0]

[*********************100%%**********************]  1 of 1 completed


35.93587875366211

In [89]:
np.random.seed(40) 

tickers = ['MSFT', 'TGT', 'QCOM', 'MU', 'CAT']
num_simulation_episodes = 500
num_simulation_days = 750

for ticker in tickers:
    data = yf.download(ticker, start="2000-01-01", end="2020-01-01")['Adj Close']
    log_returns = np.log(data / data.shift(1)).dropna()
    garch = arch_model(log_returns, p=1, q=1, dist='normal')
    res = garch.fit(disp="off")
    print(ticker)
    print(res)

[*********************100%%**********************]  1 of 1 completed
c:\Users\Biebert\miniconda3\envs\dsf\lib\site-packages\arch\univariate\base.py:311: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003602. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


[*********************100%%**********************]  1 of 1 completed
c:\Users\Biebert\miniconda3\envs\dsf\lib\site-packages\arch\univariate\base.py:311: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0004114. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y befor

MSFT
                     Constant Mean - GARCH Model Results                      
Dep. Variable:              Adj Close   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                13580.8
Distribution:                  Normal   AIC:                          -27153.6
Method:            Maximum Likelihood   BIC:                          -27127.5
                                        No. Observations:                 5030
Date:                Wed, Nov 06 2024   Df Residuals:                     5029
Time:                        12:05:52   Df Model:                            1
                                 Mean Model                                 
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu         6.9035e-04  5.161e-05     13.376  8.364e-4


c:\Users\Biebert\miniconda3\envs\dsf\lib\site-packages\arch\univariate\base.py:311: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0007068. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


[*********************100%%**********************]  1 of 1 completed
c:\Users\Biebert\miniconda3\envs\dsf\lib\site-packages\arch\univariate\base.py:311: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001308. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


[*************

MU
                     Constant Mean - GARCH Model Results                      
Dep. Variable:              Adj Close   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                10169.8
Distribution:                  Normal   AIC:                          -20331.7
Method:            Maximum Likelihood   BIC:                          -20305.6
                                        No. Observations:                 5030
Date:                Wed, Nov 06 2024   Df Residuals:                     5029
Time:                        12:05:52   Df Model:                            1
                                  Mean Model                                 
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
mu         6.2346e-04  4.001e-04      1.558      0.1


c:\Users\Biebert\miniconda3\envs\dsf\lib\site-packages\arch\univariate\base.py:311: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0004028. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.




In [90]:
res.params

mu          0.000773
omega       0.000008
alpha[1]    0.050000
beta[1]     0.930000
Name: params, dtype: float64

In [46]:
from data_scripts import generate_data as gd
from data_scripts import plotting as pl
import yfinance as yf
from arch import arch_model

np.random.seed(40) 

tickers = ['MSFT', 'TGT', 'QCOM', 'MU', 'CAT']
num_simulation_episodes = 500
num_simulation_days = 750


data = yf.download('MSFT', start="2000-01-01", end="2020-01-01")['Adj Close']
log_returns = np.log(data / data.shift(1)).dropna()
garch = arch_model(log_returns, p=1, q=1, dist='normal')
res = garch.fit(disp="off")
simulation = garch.simulate(res.params, 750)
simulation

[*********************100%%**********************]  1 of 1 completed
c:\Users\Biebert\miniconda3\envs\dsf\lib\site-packages\arch\univariate\base.py:311: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003602. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.




,data,volatility,errors
0,0.042772,0.016228,0.042082
1,0.013437,0.020397,0.012747
2,-0.003618,0.019737,-0.004309
3,-0.041948,0.018758,-0.042638
4,0.036615,0.022331,0.035925
...,...,...,...
745,0.024081,0.019122,0.023390
746,-0.037383,0.019588,-0.038073
747,0.008517,0.022131,0.007827
748,-0.014129,0.021080,-0.014819
